In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from CUP_SVM_XGBoost import xgb_fit, variable_batch_training_XGBoost, X_train, svr_fit, predict_scaled_IO_training

# Funzione per plottare RMSE
def plot_rmse_progress(train_rmse, test_rmse, title):
    fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)
    
    for i, target in enumerate(["TARGET_x", "TARGET_y", "TARGET_z"]):
        axes[i].plot(train_rmse[target], label="Train RMSE", color="blue")
        axes[i].plot(test_rmse[target], label="Test RMSE", color="red", linestyle="dashed")
        axes[i].set_title(f"{title} - {target}")
        axes[i].set_xlabel("Epochs / Mini-batches")
        axes[i].set_ylabel("RMSE")
        axes[i].legend()
        axes[i].grid(True)

    plt.tight_layout()
    plt.show()


In [ ]:
# 1. Training con Grid Search XGBoost
xgb_results, best_models_xgb, best_params_xgb, evals_result_xgb = xgb_fit()
train_rmse_xgb = {target: evals_result_xgb[target]["validation_0"]["rmse"] for target in ["TARGET_x", "TARGET_y", "TARGET_z"]}
test_rmse_xgb = {target: evals_result_xgb[target]["validation_1"]["rmse"] for target in ["TARGET_x", "TARGET_y", "TARGET_z"]}
plot_rmse_progress(train_rmse_xgb, test_rmse_xgb, "Grid Search XGBoost")

In [ ]:
# 2. Online training
#minibatch_models_online, train_rmse_online, test_rmse_online = variable_batch_training_XGBoost(best_params_xgb, batch_size=1, epochs=10)
#plot_rmse_progress(train_rmse_online, test_rmse_online, "Online Training")

In [ ]:
# 3. Mini-batch training con batch size 25
minibatch_models_25, train_rmse_25, test_rmse_25 = variable_batch_training_XGBoost(best_params_xgb, batch_size=25, epochs=10)
plot_rmse_progress(train_rmse_25, test_rmse_25, "Mini-batch Training (Batch Size = 25)")

In [ ]:
# 4. Mini-batch training con batch size 50
minibatch_models_50, train_rmse_50, test_rmse_50 = variable_batch_training_XGBoost(best_params_xgb, batch_size=50, epochs=10)
plot_rmse_progress(train_rmse_50, test_rmse_50, "Mini-batch Training (Batch Size = 50)")

In [ ]:
# 5. Mini-batch training con batch size 100
minibatch_models_100, train_rmse_100, test_rmse_100 = variable_batch_training_XGBoost(best_params_xgb, batch_size=100, epochs=10)
plot_rmse_progress(train_rmse_100, test_rmse_100, "Mini-batch Training (Batch Size = 100)")

In [ ]:
# 6. Full batch retraining
minibatch_models_full, train_rmse_full, test_rmse_full = variable_batch_training_XGBoost(best_params_xgb, batch_size=len(X_train), epochs=10)
plot_rmse_progress(train_rmse_full, test_rmse_full, "Full Batch Retraining")

In [ ]:
_, svr_best_models, _= svr_fit()

In [ ]:
def load_cup_blind_test(file_path):
    data = pd.read_csv(file_path, sep=",", comment="#", header=None)
    X = data.iloc[:, 1:13].values  # 12 feature numeriche
    return X

# Percorso del file blind
blind_testset_path = "../data/ML-CUP24-TS.csv"


# Carica i dati blind test
X_blind = load_cup_blind_test(blind_testset_path)

def predict_blind_test(model):
    return np.array([predict_scaled_IO_training(model[target], X_blind, target) for target in ["TARGET_x", "TARGET_y", "TARGET_z"]]).T 

# Creazione della figura 3D
fig = plt.figure(figsize=(18, 18))

def make_3D_plot(predictions, model_name, index):
    # Subplot per XGBoost
    ax1 = fig.add_subplot(index, projection='3d')
    ax1.scatter(predictions[:, 0], predictions[:, 1], predictions[:, 2], c='blue', marker='o', label=model_name)
    ax1.set_title("Predizioni " + model_name)
    ax1.set_xlabel("X")
    ax1.set_ylabel("Y")
    ax1.set_zlabel("Z")
    ax1.legend()
    ax1.grid()

make_3D_plot(predict_blind_test(svr_best_models),"SVR retrained full batch", 331)
make_3D_plot(predict_blind_test(best_models_xgb),"XGBoost retrained full batch", 332)
make_3D_plot(predict_blind_test(minibatch_models_25),"XGBoost retrained batchsize 25", 333)
make_3D_plot(predict_blind_test(minibatch_models_50),"XGBoost retrained batchsize 50", 334)
make_3D_plot(predict_blind_test(minibatch_models_100),"XGBoost retrained batchsize 100", 335)


plt.show()

In [ ]:
from CUP_SVM_XGBoost import scaler_input

def load_cup_blind_test(file_path):
    data = pd.read_csv(file_path, sep=",", comment="#", header=None)
    X = data.iloc[:, 1:13].values  # 12 feature numeriche
    return X

# Percorso del file di training
blind_testset_path = "../data/ML-CUP24-TS.csv"


# Carica i dati blind test
X_blind = load_cup_blind_test(blind_testset_path)


def predict_blind_test(model):
    return np.array([model[target].predict(scaler_input.transform(X_blind)) for target in ["TARGET_x", "TARGET_y", "TARGET_z"]]).T 


# Creazione della figura 3D
fig = plt.figure(figsize=(18, 18))

def make_3D_plot(predictions, model_name, index):
    # Subplot per XGBoost
    ax1 = fig.add_subplot(index, projection='3d')
    ax1.scatter(predictions[:, 0], predictions[:, 1], predictions[:, 2], c='blue', marker='o', label=model_name)
    ax1.set_title("Predizioni " + model_name)
    ax1.set_xlabel("X")
    ax1.set_ylabel("Y")
    ax1.set_zlabel("Z")
    ax1.legend()
    ax1.grid()

make_3D_plot(predict_blind_test(svr_best_models),"SVR retrained full batch", 331)
make_3D_plot(predict_blind_test(best_models_xgb),"XGBoost retrained full batch", 332)
make_3D_plot(predict_blind_test(minibatch_models_25),"XGBoost retrained batchsize 25", 333)
make_3D_plot(predict_blind_test(minibatch_models_50),"XGBoost retrained batchsize 50", 334)
make_3D_plot(predict_blind_test(minibatch_models_100),"XGBoost retrained batchsize 100", 335)